Fist of all we need to get the Python tools in the environment.

At the moment this step is a bit heavy, but it will be simplified in the future.

In [1]:
import sys
import os
if not os.path.exists('LbNightlyTools'):
    !git clone -q ssh://git@gitlab.cern.ch:7999/lhcb-core/LbNightlyTools.git
sys.path.insert(0, os.path.join(os.getcwd(), 'LbNightlyTools', 'python'))
os.environ['PATH'] = os.pathsep.join([os.path.join(os.getcwd(), 'LbNightlyTools', 'scripts'), os.environ['PATH']])

This hack is needed to avoid tons of messages in IPython notebook.

In [2]:
import logging
root_logger = logging.getLogger()
root_logger.setLevel(logging.INFO)
if root_logger.handlers:
    root_logger.handlers[0].setLevel(root_logger.level)

Now let's get the required Python classes.

In [3]:
from LbNightlyTools.Configuration import *

For a local build of a few interdependent projects we need to create a `Slot` instance that groups the projects.

In [4]:
slot = Slot('my-test', projects=[Project('Gaudi', 'v26r3b'), Project('LHCb', 'HEAD')])
slot.cache_entries['CMAKE_USE_CCACHE'] = True # enable ccache use in the builds

Now we are ready to checkout the code.

In [5]:
out = slot.checkout()

INFO:LbNightlyTools.Configuration:checking out Gaudi v26r3b
INFO:LbNightlyTools.Configuration:checking out LHCb HEAD


A plain checkout of the projects cannot be used out of the box: we need to patch the configuration files so that the projects form a consistent set.

In [6]:
slot.patch()

INFO:LbNightlyTools.Configuration:patching GAUDI/GAUDI_v26r3b/CMakeLists.txt
INFO:LbNightlyTools.Configuration:patching GAUDI/GAUDI_v26r3b/cmt/project.cmt
INFO:LbNightlyTools.Configuration:patching GAUDI/GAUDI_v26r3b/GaudiRelease/cmt/requirements
INFO:LbNightlyTools.Configuration:patching LHCB/LHCB_HEAD/CMakeLists.txt
INFO:LbNightlyTools.Configuration:patching LHCB/LHCB_HEAD/cmt/project.cmt
INFO:LbNightlyTools.Configuration:patching LHCB/LHCB_HEAD/LHCbSys/cmt/requirements


Now it's possible to build (e.g. with 4 parallel jobs).

In [7]:
out = slot.build(jobs=4)

INFO:LbNightlyTools.Configuration:building Gaudi v26r3b
INFO:LbNightlyTools.Configuration:building LHCb HEAD


The output of checkout and build is kept in data members of each project: `checkout_log` and `build_log`.

In [8]:
# print the last 10 lines of the build log of Gaudi
print('\n'.join(slot.Gaudi.build_log.splitlines()[-10:]))


make[4]: Leaving directory `/afs/cern.ch/work/m/marcocle/ipynb/GAUDI/GAUDI_v26r3b/build.x86_64-slc6-gcc48-opt'
/afs/cern.ch/sw/lcg/contrib/CMake/2.8.9/Linux-i386/bin/cmake -E cmake_progress_report /afs/cern.ch/work/m/marcocle/ipynb/GAUDI/GAUDI_v26r3b/build.x86_64-slc6-gcc48-opt/CMakeFiles
Built target post-install
make[3]: Leaving directory `/afs/cern.ch/work/m/marcocle/ipynb/GAUDI/GAUDI_v26r3b/build.x86_64-slc6-gcc48-opt'
/afs/cern.ch/sw/lcg/contrib/CMake/2.8.9/Linux-i386/bin/cmake -E cmake_progress_start /afs/cern.ch/work/m/marcocle/ipynb/GAUDI/GAUDI_v26r3b/build.x86_64-slc6-gcc48-opt/CMakeFiles 0
make[2]: Leaving directory `/afs/cern.ch/work/m/marcocle/ipynb/GAUDI/GAUDI_v26r3b/build.x86_64-slc6-gcc48-opt'
make[1]: Leaving directory `/afs/cern.ch/work/m/marcocle/ipynb/GAUDI/GAUDI_v26r3b/build.x86_64-slc6-gcc48-opt'
command exited with code 0
Completed at: 2015-10-01 17:02:17.447165
Elapsed time: 0:06:34.213523


In [9]:
# print the last 10 lines of the build log of LHCb
print('\n'.join(slot.LHCb.build_log.splitlines()[-10:]))

make[4]: Leaving directory `/afs/cern.ch/work/m/marcocle/ipynb/LHCB/LHCB_HEAD/build.x86_64-slc6-gcc48-opt'
/afs/cern.ch/sw/lcg/contrib/CMake/2.8.9/Linux-i386/bin/cmake -E cmake_progress_report /afs/cern.ch/work/m/marcocle/ipynb/LHCB/LHCB_HEAD/build.x86_64-slc6-gcc48-opt/CMakeFiles
Built target post-install
make[3]: Leaving directory `/afs/cern.ch/work/m/marcocle/ipynb/LHCB/LHCB_HEAD/build.x86_64-slc6-gcc48-opt'
/afs/cern.ch/sw/lcg/contrib/CMake/2.8.9/Linux-i386/bin/cmake -E cmake_progress_start /afs/cern.ch/work/m/marcocle/ipynb/LHCB/LHCB_HEAD/build.x86_64-slc6-gcc48-opt/CMakeFiles 0
make[2]: Leaving directory `/afs/cern.ch/work/m/marcocle/ipynb/LHCB/LHCB_HEAD/build.x86_64-slc6-gcc48-opt'
make[1]: Leaving directory `/afs/cern.ch/work/m/marcocle/ipynb/LHCB/LHCB_HEAD/build.x86_64-slc6-gcc48-opt'
command exited with code 0
Completed at: 2015-10-01 17:27:48.132672
Elapsed time: 0:25:30.680800


Running the tests is simple too.

In [10]:
# - the keword argument `args` is a list of arguments to make
# - the make argument `ARGS` is passed to ctest (see `ctest --help`)
out = slot.Gaudi.test(args=['ARGS=-L GaudiKernel'])
print(slot.Gaudi.test_log)

INFO:LbNightlyTools.Configuration:testing Gaudi v26r3b


Started at: 2015-10-01 17:27:48.536012
running make ARGS=-L GaudiKernel configure
cd build.x86_64-slc6-gcc48-opt && make rebuild_cache
make[1]: Entering directory `/afs/cern.ch/work/m/marcocle/ipynb/GAUDI/GAUDI_v26r3b/build.x86_64-slc6-gcc48-opt'
Running CMake to regenerate build system...
lbn-wrapcmd /afs/cern.ch/work/m/marcocle/ipynb/GAUDI/GAUDI_v26r3b/build.x86_64-slc6-gcc48-opt rebuild_cache  /afs/cern.ch/sw/lcg/contrib/CMake/2.8.9/Linux-i386/bin/cmake -H/afs/cern.ch/work/m/marcocle/ipynb/GAUDI/GAUDI_v26r3b -B/afs/cern.ch/work/m/marcocle/ipynb/GAUDI/GAUDI_v26r3b/build.x86_64-slc6-gcc48-opt
Re-run cmake no build system arguments
-- Target system: x86_64-slc6-gcc48
-- Build type: opt
-- Host system: x86_64-slc6-gcc44
-- Using heptools 79 from /afs/cern.ch/sw/lcg/releases/LCG_79/LCG_externals_x86_64-slc6-gcc48-opt.txt
-- Using ccache for building
-- Prefix COMPILE commands with 'lbn-wrapcmd <CMAKE_CURRENT_BINARY_DIR> <TARGET_NAME> /usr/bin/ccache'
-- Prefix LINK commands with 'lbn-wra